In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions

In [5]:
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *

from pyspark.ml  import Pipeline     
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#create session in order to be capable of accessing all Spark API
spark = SparkSession \
    .builder \
    .appName("Purchase") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [10]:
customerDataframe = spark.read.csv(
    "D:/kt/datasets/customer_data/Mall_Customers.csv", 
    header=True)

In [11]:
customerDataframe.show(3)

+----------+------+---+------------------+--------------+
|CustomerID| Genre|Age|Annual_Income_(k$)|Spending_Score|
+----------+------+---+------------------+--------------+
|         1|  Male| 19|                15|            39|
|         2|  Male| 21|                15|            81|
|         3|Female| 20|                16|             6|
+----------+------+---+------------------+--------------+
only showing top 3 rows



In [12]:
#count number of rows of our dataFrame
num_rows = customerDataframe.count()
print("number of rows: ", num_rows)

number of rows:  200


In [13]:
#show our dataFrame schema
customerDataframe.printSchema()

root
 |-- CustomerID: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Annual_Income_(k$): string (nullable = true)
 |-- Spending_Score: string (nullable = true)



In [15]:
#show statistic of the data we want
customerDataframe.describe('Spending_Score').show()

+-------+------------------+
|summary|    Spending_Score|
+-------+------------------+
|  count|               200|
|   mean|              50.2|
| stddev|25.823521668370173|
|    min|                 1|
|    max|                99|
+-------+------------------+



In [16]:
customerDataframe.select('Genre').distinct().count()

2

In [17]:
#create new dataFrame from "City" and "Total" columns
newDataframe = customerDataframe.select(customerDataframe['Genre'], 
                                              customerDataframe['Age'])

# top 10 rows 
newDataframe.show(5); 

print('=========================')
# schema of dataframe
newDataframe.printSchema() 

+------+---+
| Genre|Age|
+------+---+
|  Male| 19|
|  Male| 21|
|Female| 20|
|Female| 23|
|Female| 31|
+------+---+
only showing top 5 rows

root
 |-- Genre: string (nullable = true)
 |-- Age: string (nullable = true)



In [18]:
#filter only row data whose "Total" column value > 300
customerDataframe.filter(customerDataframe['Spending_Score'] > 30).show(5)

+----------+------+---+------------------+--------------+
|CustomerID| Genre|Age|Annual_Income_(k$)|Spending_Score|
+----------+------+---+------------------+--------------+
|         1|  Male| 19|                15|            39|
|         2|  Male| 21|                15|            81|
|         4|Female| 23|                16|            77|
|         5|Female| 31|                17|            40|
|         6|Female| 22|                17|            76|
+----------+------+---+------------------+--------------+
only showing top 5 rows



In [19]:
# sorting dataframe by Age
sortedByAge = customerDataframe.orderBy('Age').show(10)

+----------+------+---+------------------+--------------+
|CustomerID| Genre|Age|Annual_Income_(k$)|Spending_Score|
+----------+------+---+------------------+--------------+
|        66|  Male| 18|                48|            59|
|        92|  Male| 18|                59|            41|
|        34|  Male| 18|                33|            92|
|       115|Female| 18|                65|            48|
|        69|  Male| 19|                48|            59|
|       112|Female| 19|                63|            54|
|         1|  Male| 19|                15|            39|
|       114|  Male| 19|                64|            46|
|       116|Female| 19|                65|            50|
|        62|  Male| 19|                46|            55|
+----------+------+---+------------------+--------------+
only showing top 10 rows



In [20]:
numTransactionEachAge = customerDataframe.groupBy("Age").count()
numTransactionEachAge.show(5)

+---+-----+
|Age|count|
+---+-----+
| 51|    2|
| 54|    4|
| 29|    5|
| 69|    1|
| 42|    2|
+---+-----+
only showing top 5 rows



In [21]:
from pyspark.sql.types import IntegerType,BooleanType,DateType


DataFrame[Age_customer: int]

In [24]:
df = customerDataframe.withColumn("Age", 
                                  customerDataframe["Age"]
                                  .cast('float'))
df.printSchema()

root
 |-- CustomerID: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Age: float (nullable = true)
 |-- Annual_Income_(k$): string (nullable = true)
 |-- Spending_Score: string (nullable = true)



In [25]:
meanTotal = df.groupBy().avg("Age").take(1)[0][0]
print('Mean total:',meanTotal)

Mean total: 38.85
